<a href="https://colab.research.google.com/github/mishabar410/ML-2022/blob/main/TransAttUnet/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torch import nn
improt torch.functional as F

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in, out):
        super().__init__()

        self.conv1 = nn.Conv2d(in, out, kernel_size=3)
        self.bnorm1 = nn.BatchNorm2d(out)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out, out, kernel_size=3)
        self.bnorm2 = nn.BatchNorm2d(out)
    
    def forward(self, x)
        x1 = self.relu(self.bnorm1(self.conv1(x)))
        x2 = self.relu(self.bnorm2(self.conv2(x)))
        return x2

class Down(nn.Module):
    def __init__(self, in, out):
        self.conv = DoubleConv(in, out)
        self.mpool = self.MaxPool2d(kernel_size = 3, stride = 2)

    def forward(self, x):
        return self.conv(self.mpool(x))

class Up(nn.Module):
    def __init__(self, in, out):
        self.conv = DoubleConv(in, out)
        self.upsample = nn.Upsample(scale_factor = 2, mode = 'bilinear')

    def forward(self, x1, x2):
        x1 = self.upsaple(x1)
        x1 = torch.cat([x1, x2], 1)
        return self.conv(x1)

In [ ]:
class GSA(nn.Module):
    def __init__(self):
        self.K_conv = nn.Conv2d(in, in // 8)
        self.Q_conv = nn.Conv2d(in, in // 8)
        self.W_conv = nn.Conv2d(in, in)
        self.lambda = torch.parameter(torch.zeros(1))
    
    def forward(x):
        batch_size, channels, height, width = x.size()
        M = self.M_conv(x).view(batch_size, -1, height * width)
        N = self.N_conv(x).view(batch_size, -1, height * width).permute(0, 2, 1)
        W = self.W_conv(x).view(batch_size, -1, height * width)
        B = F.softmax(torch.matmul(M, N)).permute(0, 2, 1).permute(0, 2, 1)
        result = torch.matmul(W, B).view(batch_size, channels, height, width)
        
        return self.lambda * result + x